Package Installation

In [ ]:
! pip install azure-search-documents==11.6.0b4 --quiet
! pip install azure-identity --quiet
! pip install openai --quiet

Set API Keys

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

# Fill in these values with your own
endpoint = ""
credential = AzureKeyCredential("")
index_name = ""
azure_openai_endpoint = ""
azure_openai_key = ""
azure_openai_embedding_deployment = ""
azure_openai_embedding_dimensions = 1024
embedding_model_name = ""
azure_openai_api_version = "2024-06-01"

Generate Embeddings

In [ ]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    # Fill this in
)

path = os.path.join('television-data.json')
with open(path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

titles = [item['title'] for item in input_data]
content = [item['content'] for item in input_data]
title_response = client.embeddings.create(input=titles, model=embedding_model_name, dimensions=azure_openai_embedding_dimensions)
title_embeddings = [item.embedding for item in title_response.data]
content_response = client.embeddings.create(input=content, model=embedding_model_name, dimensions=azure_openai_embedding_dimensions)
content_embeddings = [item.embedding for item in content_response.data]

for i, item in enumerate(input_data):
    title = item['title']
    content = item['content']
    item['titleVector'] = title_embeddings[i]
    item['contentVector'] = content_embeddings[i]

output_path = os.path.join('docVectors.json')

with open(output_path, "w") as f:
    json.dump(input_data, f)

Setup Fields

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)

index_client = SearchIndexClient(
    endpoint=endpoint, credential=credential)
fields = [
    # Fill this in
]

Configure the Vector Search

In [ ]:
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer="myVectorizer"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="myVectorizer",
            azure_open_ai_parameters=AzureOpenAIParameters(
                # Fill this in
            )
        )
    ]
)

Configure the Semantic Search

In [ ]:

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        # Fill this in
    )
)

semantic_search = SemanticSearch(default_configuration_name = "my-semantic-config",  configurations=[semantic_config])

Create the Search Index

In [ ]:
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

Upload to Service

In [ ]:
from azure.search.documents import SearchClient
import json

output_path = os.path.join('docVectors.json')

with open(output_path, 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Vector Search

In [ ]:
from azure.search.documents.models import VectorizedQuery

query = ""  # Fill this in
embedding = client.embeddings.create(input=query, model=embedding_model_name, dimensions=azure_openai_embedding_dimensions).data[0].embedding

vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")

Cleanup

In [ ]:
index_client.delete_index(index)